In [3]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from pybaseball import statcast, cache
#from catboost import CatBoostClassifier
from xgboost import XGBClassifier
import joblib
import math
from pybaseball import statcast
import scipy.stats as stats
#from catboost import Pool
#import optuna
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score, precision_score
import sqlite3
cache.enable()

In [4]:
data = pd.read_csv('21-24pitching.csv')

In [7]:
data['release_speed'].mean()

np.float64(88.98224756397475)

In [35]:
data = data[~data['pitch_name'].isin(['Pitch Out', 'Eephus', 'Knuckleball'])]
data.loc[data['p_throws'] == 'L', 'pfx_x'] *= -1
data.loc[data['p_throws'] == 'L', 'release_pos_x'] *= -1
data.loc[data['p_throws'] == 'L', 'spin_axis'] = 360 - data.loc[data['p_throws'] == 'L', 'spin_axis']
fastballs = ['4-Seam Fastball', 'Sinker']
offspeed = ['Split-Finger', 'Changeup', 'Forkball']
breaking = ['Curveball', 'Slider', 'Cutter', 'Knuckle Curve', 'Sweeper', 'Screwball', 'Slow Curve']
print(data)

        pitch_type   game_date  release_speed  release_pos_x  release_pos_z  \
0               KC  2024-10-30           77.5          -1.11           5.65   
1               KC  2024-10-30           78.7          -1.01           5.73   
2               FC  2024-10-30           93.1          -1.19           5.53   
3               KC  2024-10-30           78.5          -1.19           5.70   
4               KC  2024-10-30           77.4          -1.23           5.78   
...            ...         ...            ...            ...            ...   
2990317         FF  2021-03-15           90.1          -2.72           6.10   
2990318         SI  2021-03-15           90.3          -2.71           6.05   
2990319         SI  2021-03-15           91.0          -2.93           6.07   
2990320         CH  2021-03-15           78.2          -2.45           6.17   
2990321         SL  2021-03-15           83.7          -2.48           6.17   

             player_name  batter  pitcher     event

In [18]:
def get_pitch_data(data, pitch_name, players):
    pitch_data = data.loc[(data['pitch_name'] == pitch_name) & (data['player_name'].isin(players))]
    grouped_data = pitch_data.groupby('player_name').agg({
        'release_speed': 'median',
        'pfx_x': 'median',
        'pfx_z': 'median'
    }).reset_index()
    
    return grouped_data

In [19]:
all = data['player_name'].unique()

# pitch type: Fastballs

fastball_data = get_pitch_data(data, '4-Seam Fastball', all)
fastball_data = fastball_data.dropna(subset=['release_speed'])
fastball_velo = dict(zip(fastball_data['player_name'], fastball_data['release_speed']))
fastball_horiz = dict(zip(fastball_data['player_name'], fastball_data['pfx_x']))
fastball_vert = dict(zip(fastball_data['player_name'], fastball_data['pfx_z']))

sinker_data = get_pitch_data(data, 'Sinker', all)
sinker_data = sinker_data.dropna(subset=['release_speed'])
sinker_velo = dict(zip(sinker_data['player_name'], sinker_data['release_speed']))
sinker_horiz = dict(zip(sinker_data['player_name'], sinker_data['pfx_x']))
sinker_vert = dict(zip(sinker_data['player_name'], sinker_data['pfx_z']))

cutter_data = get_pitch_data(data, 'Cutter', all)
cutter_data = cutter_data.dropna(subset=['release_speed'])
cutter_velo = dict(zip(cutter_data['player_name'], cutter_data['release_speed']))
cutter_horiz = dict(zip(cutter_data['player_name'], cutter_data['pfx_x']))
cutter_vert = dict(zip(cutter_data['player_name'], cutter_data['pfx_z']))

In [24]:
pitch_data = {
    '4-Seam Fastball': {
        'velo': fastball_velo,
        'horiz': fastball_horiz,
        'vert': fastball_vert
    },
    'Sinker': {
        'velo': sinker_velo,
        'horiz': sinker_horiz,
        'vert': sinker_vert
    },
    'Cutter': {
        'velo': cutter_velo,
        'horiz': cutter_horiz,
        'vert': cutter_vert
    }
}

def calculate_differences(row):
    player_name = row['player_name']
    for pitch_type in ['4-Seam Fastball', 'Sinker', 'Cutter']:
        if player_name in pitch_data[pitch_type]['velo']:
            velo_dif = row['release_speed'] - pitch_data[pitch_type]['velo'][player_name]
            horiz_dif = row['pfx_x'] - pitch_data[pitch_type]['horiz'][player_name]
            vert_dif = row['pfx_z'] - pitch_data[pitch_type]['vert'][player_name]
            return velo_dif, horiz_dif, vert_dif

    return np.nan, np.nan, np.nan

data['velo'], data['horiz_dif'], data['vert_dif'] = zip(*data.apply(calculate_differences, axis=1))

In [52]:
league_avg_run_exp = data['delta_run_exp'].mean()
clase = data[data['player_name'] == "Clase, Emmanuel"]
clase_cutter = clase[clase['pitch_name'] == 'Cutter']
clase_cutter_run_exp = clase_cutter['delta_run_exp'].mean()
clase_cutter_stuff = 100 + ((clase_cutter_run_exp/league_avg_run_exp) / 100)
print(clase_cutter_stuff)

burnes = data[data['player_name'] == "Burnes, Corbin"]
clase_cutter = burnes[burnes['pitch_name'] == 'Cutter']
clase_cutter_run_exp = clase_cutter['delta_run_exp'].mean()
clase_cutter_stuff = 100 + ((clase_cutter_run_exp/league_avg_run_exp) / 100)
print(clase_cutter_stuff)

107.90454176810843
106.01141760315578
